In [13]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [14]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [15]:
# default_exp agents.dqn.targets

In [16]:
# export
# Python native modules
import os
from collections import deque
from typing import *
# Third party libs
import torch
from torch.nn import *
from fastcore.all import *
from fastai.learner import *
from fastai.torch_basics import *
from fastai.torch_core import *
from fastai.callback.all import *
# Local modules
from fastrl.data.block import *
from fastrl.data.gym import *
from fastrl.agent import *
from fastrl.core import *
from fastrl.agents.dqn.core import *
from fastrl.memory.experience_replay import *

# DQN Targets + N-Step
> A Bare-Bones DQN is usually extremely unstable. Target models can eleviate this. We also support First-Last N steps better.

In [17]:
GAMMA=0.99
def calc_target(net, local_reward, next_state,d):
    if next_state is None or d:
        return local_reward
    state_v = torch.tensor([next_state], dtype=torch.float32).to(default_device())
    next_q_v = net(state_v)
    best_q = next_q_v.max(dim=1)[0].item()
    return local_reward + GAMMA * best_q



In [18]:
# # for  i in range(1000):

# # learn.opt.zero_grad()

# learn.state_action_values = learn.model.model(learn.xb['state']).gather(1, learn.xb['action']).squeeze(-1)
# learn.next_state_values = learn.target_model(learn.xb['next_state']).max(1)[0]
# learn.next_state_values[learn.xb['done'].squeeze(-1)] = 0.0

# learn.expected_state_action_values = learn.next_state_values.detach() * (0.99**3) + learn.xb['reward'].squeeze(-1)
# learn.loss= nn.MSELoss()(learn.state_action_values,learn.next_state_values)

# # learn.loss.backward()
# # learn.opt.step()

# print(learn.loss)

In [20]:
# learn.state_action_values

In [21]:
# learn.xb['reward'].squeeze(-1)

In [22]:
# learn.next_state_values.detach()

In [59]:
# export
class DQNTargetTrainer(Callback):
    
    def __init__(self,n_batch=0,target_sync=300,discount=0.99,n_steps=1):
        store_attr()
        self._xb=None  
        
    def before_fit(self):
        self.learn.target_model=deepcopy(self.learn.model.model)
        self.n_batch=0
    
    def after_pred(self):
        self._xb=self.yb
        self.learn.yb=[]
        

  
        self.learn.opt.zero_grad()

        self.learn.state_action_values = self.learn.model.model(self.learn.xb['state']).gather(1, self.learn.xb['action']).squeeze(-1)
        with torch.no_grad():
            self.learn.next_state_values = self.target_model(self.xb['next_state']).max(1)[0]
            self.learn.next_state_values[self.xb['done'].squeeze(-1)] = 0.0

            self.learn.expected_state_action_values = self.learn.next_state_values.detach() * (self.discount**self.n_steps) + self.xb['reward'].squeeze(-1)
        self.learn.loss= nn.MSELoss()(self.learn.state_action_values,self.learn.expected_state_action_values)
        
        if (self.n_batch-1)%self.target_sync==0:
            print('The loss should be practically zero: ',self.loss)
            print(self.learn.state_action_values-self.learn.expected_state_action_values)
        
        # raise Exception
        self.learn.loss.backward()
        self.learn.opt.step()
#         self.learn.batch_targets = torch.cat([calc_target(self.learn.model.model, r, ns.cpu().numpy(),d)
#                          for r,ns,d in zip(self.learn.xb['reward'],self.learn.xb['next_state'],self.learn.xb['done'])])
        
#         self.learn.opt.zero_grad()
#         self.learn.states_v = self.xb['state'].to(default_device()).float()
#         self.learn.net_q_v = self.learn.model.model(self.learn.states_v)
#         # print(net_q_v)
#         self.learn.target_q = self.learn.net_q_v.cpu().data.numpy().copy()
        
#         # print(batch_targets,target_q)
#         self.learn.target_q[range(self.learn.xb.bs()), self.xb['action'].cpu()] = self.learn.batch_targets.cpu()
#         self.learn.target_q_v = torch.tensor(self.learn.target_q)
#         # print(net_q_v, target_q_v)
#         loss_v = self.learn.loss_func(self.learn.net_q_v.cpu(), self.learn.target_q_v.cpu())
#         loss_v.backward()
#         self.learn.loss=loss_v.cpu()
#         # print(loss_v)
#         self.learn.opt.step()
        
#         self.learn.yb=self.xb
#         self.learn.xb=self.xb[0]
#         self._xb=({k:v.clone() for k,v in self.xb.items()},)
#         self.learn.done_mask=self.xb['done'].reshape(-1,)
#         self.learn.next_q=self.target_model(self.xb['next_state']).max(dim=1).values.reshape(-1,1)
#         self.learn.next_q[self.done_mask]=1
#         self.learn.targets=self.xb['reward']+self.learn.next_q*(self.discount**self.n_steps)
#         self.learn.pred=self.learn.model.model(self.xb['state'])

#         t_q=self.pred.clone()
#         t_q.scatter_(1,self.xb['action'],self.targets)
#         self.learn.yb=(t_q,)
        with torch.no_grad():
            self.learn.td_error=(self.learn.state_action_values.cpu()-self.learn.expected_state_action_values.cpu()).reshape(-1,1)**2
        
    def before_backward(self): self.learn.yb=self._xb
        
    def after_batch(self):
        if self.n_batch%self.target_sync==0:
            self.target_model.load_state_dict(self.learn.model.state_dict())
            # if self.n_batch>1:raise Exception
        self.n_batch+=1

In [60]:
dqn=DQN(4,2)
agent=Agent(dqn,cbs=[ArgMaxFeed,DiscreteEpsilonRandomSelect(min_epsilon=0.02)])
source=Source(cbs=[GymLoop('CartPole-v1',agent,steps_count=1,seed=None,#mode='rgb_array',
                           steps_delta=1),FirstLast#,ResReduce(reduce_by=4)
                  ])
dls=SourceDataBlock().dataloaders([source],n=1000,bs=1,num_workers=0)

er_tb=ExperienceReplayTensorboard(comment='_dqn_target',every_epoch=1)
# opt=optim.Adam(dqn.parameters(), lr=0.0001)
learn=Learner(dls,agent,loss_func=MSELoss(),
              opt_func=partial(OptimWrapper,dqn.parameters(),optim.Adam),
              cbs=[ExperienceReplayCallback(bs=32,max_sz=1000,warmup_sz=32,freeze_at_max=True),
                   DQNTargetTrainer(n_steps=1,target_sync=1000)
                   ,er_tb
                  ],
              metrics=[Reward,Epsilon,NEpisodes])

Could not do one pass in your dataloader, there is something wrong in it


In [61]:
# hide
SHOW_TENSOR_BOARD=True
if not os.environ.get("IN_TEST", None) and SHOW_TENSOR_BOARD:
    run_tensorboard(samples_per_plugin='images=2000')

A few things, the loss should be <1. If it is not, there is something majorly wrong iwth the training.

It is alright for the actual values twe are comparing to be way more than >1 though, but it is expected that the loss should not be all that high at all.

In [62]:
slow=True
learn.fit(3 if not slow else 47,lr=0.0001,wd=None)

/home/fastrl_user/fastrl/fastrl/memory/experience_replay.py:134: UserWarning: image is missing from the experience replay. Image section of the replay will not be logged.
  warn('image is missing from the experience replay. Image section of the replay will not be logged.')


The loss should be practically zero:  TensorBatch(0.9131, device='cuda:0', grad_fn=<AliasBackward>)
TensorBatch([-1.0310, -1.0174, -1.0335, -1.0140, -1.0262, -1.0365,  0.0743, -1.0169,
        -1.0338, -1.0237, -1.0161,  0.1999, -1.0134, -1.0008, -1.0263, -1.0111,
        -1.0202, -1.0342,  0.0942, -1.0017, -1.0183, -1.0153,  0.1711, -1.0440,
        -1.0285, -0.9961, -1.0146, -0.9911, -1.0084, -1.0213, -1.0359, -1.0289],
       device='cuda:0', grad_fn=<AliasBackward>)
The loss should be practically zero:  TensorBatch(0.9731, device='cuda:0', grad_fn=<AliasBackward>)
TensorBatch([-1.0125, -0.9969, -0.9906, -0.9892, -0.9642, -1.0204, -0.9826, -1.0040,
        -0.9067, -1.0182, -0.9849, -0.9459, -1.0128, -0.9545, -1.0022, -1.0113,
        -1.0066, -1.0112, -0.9912, -0.9975, -0.9502, -1.0180, -1.0112, -1.0097,
        -1.0121, -1.0156, -0.9208, -0.9901,  1.0132,  0.7348, -1.0257, -1.0165],
       device='cuda:0', grad_fn=<AliasBackward>)
The loss should be practically zero:  TensorBatch(

KeyboardInterrupt: 

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()